## Imports

In [ ]:
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import decode_predictions
import cv2
from skimage import io, transform, segmentation

## Data

In [ ]:
image = io.imread('images/input/Maula.jpg')
target_image_size = (299, 299)
image = transform.resize(image, target_image_size)
plt.imshow(image)

In [ ]:
image.shape

## Model

In [ ]:
# model = ResNet50()
model = InceptionV3() #Load pretrained model

In [ ]:
# model.summary()
# Inception preprocessing
image = (image - 0.5)*2
preds = model.predict(image[np.newaxis, ...])
print(decode_predictions(preds))

## Grad-CAM
As a summary:
- Step 1: The model is broken into to parts. The first part goes from the inputs to the the last convolutional layer (last_conv_layer_model). The second part goes from the last convolutional layer to the prediction (classifier_model).
- Step 2: The image goes to the first part and then it starts being watched.
- Step 3: After that, $\alpha_k$ is computed for all the filters in the last conv layer.
- Step 4: Finally, the activation map is obtained.

### Step 1

We get the output of the last convolution layer.

In [ ]:
# model.summary()
last_conv_layer_name = 'mixed10'
last_conv_layer = model.get_layer(last_conv_layer_name)

We create a model that goes up to only that layer.

In [ ]:
last_conv_layer_model = tf.keras.Model(model.inputs, last_conv_layer.output)

In [ ]:
list_layers = []
last_conv_layer_found = False
for layer in model.layers:
    current_layer_name = layer.name
    if current_layer_name == last_conv_layer_name:
        last_conv_layer_found = True
    if last_conv_layer_found and current_layer_name != last_conv_layer_name:
        list_layers.append(current_layer_name)

We create a model which takes the output of the model above and uses the remaining layers to get the final predictions.

In [ ]:
classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
x = classifier_input
for idx, layer_name in enumerate(list_layers): # These are the remaining layers in the ResNet50 model
    x = model.get_layer(layer_name)(x)
classifier_model = tf.keras.Model(classifier_input, x)

### Step 2

- We get the output from the model up till the last convolution layer.
- We ask tf to watch this tensor output, as we want to calculate the gradients of the predictions of our target class wrt to the output of this model (last convolution layer model).

In [ ]:
with tf.GradientTape() as tape:
    inputs = image[np.newaxis, ...] #adding axis (batch)
    last_conv_layer_output = last_conv_layer_model(inputs) #from beginning of RestNet to the last conv layer 
    tape.watch(last_conv_layer_output) # Start watching this part
    preds = classifier_model(last_conv_layer_output) # Make a prediction
    top_pred_index = tf.argmax(preds[0]) #Get the class of the highest probability
    top_class_channel = preds[:, top_pred_index]

We compute the gradient with respect to the last conv layer. Grads contains the following information:
<h3 align="center">$\frac{\partial y}{\partial A_{ij}^k}$</h3> 

In [ ]:
grads = tape.gradient(top_class_channel, last_conv_layer_output)

In [ ]:
print(grads.shape)
print(last_conv_layer.output_shape)

We compute $\alpha_k = \frac{1}{Z} \sum_i \sum_j \frac{\partial y}{\partial A_{ij}^k}$

In [ ]:
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

In [ ]:
print(pooled_grads.shape)

The next step is to compute $S = \sum \alpha_k A^k$

In [ ]:
output = last_conv_layer_output.numpy()[0]
pooled_grads = pooled_grads.numpy()
for i in range(pooled_grads.shape[-1]):
    output[:, :, i] *= pooled_grads[i]

In [ ]:
gradcam = np.sum(output, axis=-1)

Finally, we apply $ReLU$ function to S: $L_{grad-CAM} = ReLU(s)$

In [ ]:
gradcam = np.clip(gradcam, 0, np.max(gradcam))
gradcam = cv2.resize(gradcam, target_image_size)

In [ ]:
plt.imshow(image)
plt.imshow(gradcam, alpha=0.5)

In [ ]:
fig = plt.gcf()
DPI = fig.get_dpi()
fig.set_size_inches(299.0/float(DPI),299.0/float(DPI))
plt.imshow(image)
plt.imshow(gradcam, alpha=0.5)
plt.axis('off')
fig.savefig('images/output/maula_gradcam.png')